In [1]:
import pandas as pd

In [2]:
pd.read_csv('/glade/work/guoqiang/CTSM_CAMELS/data_mesh_surf/HillslopeHydrology/CAMELS_level1_basin_info.csv')

,hru_id,ann_P,lon_cen,lat_cen,AREA,elev_mean,ave_T,july_T,Perimeter,Source,...,RawTDXarea,HSTDXarea,MHTDXarea,RawCAMELSbias,RawTDXbias,HSTDXbias,MHTDXbias,final_bias,geometry,file_obsQ
0,1013500,0.0,-68.56551,47.01169,2.303988e+09,277.493490,NaN,NaN,647993,RawTDX,...,2246.946668,2246.628415,2246.132802,0.022767,-0.002554,-0.002695,-0.002915,0.002554,POLYGON ((-68.84000529335947 46.73746031546882...,/glade/work/guoqiang/CTSM_cases/CAMELS_Calib/C...
1,1022500,0.0,-68.07313,44.79691,6.203873e+08,103.604200,NaN,NaN,312624,HSTDX,...,615.173865,586.427404,586.602450,0.081568,0.072479,0.022363,0.022668,0.022363,POLYGON ((-68.13005554673525 44.64716666195261...,/glade/work/guoqiang/CTSM_cases/CAMELS_Calib/C...
2,1030500,0.0,-68.14985,45.80527,3.676155e+09,174.433896,NaN,NaN,662248,RawCAMELS,...,3654.475429,3653.788946,3653.845830,-0.000004,-0.005901,-0.006088,-0.006073,0.000004,POLYGON ((-68.35800170861474 45.67902755716805...,/glade/work/guoqiang/CTSM_cases/CAMELS_Calib/C...
3,1031500,0.0,-69.58119,45.23568,7.665447e+08,304.743490,NaN,NaN,309614,RawCAMELS,...,775.632787,772.001421,772.168545,-0.003258,0.008560,0.003838,0.004055,0.003258,POLYGON ((-69.67646789509475 45.14558029212971...,/glade/work/guoqiang/CTSM_cases/CAMELS_Calib/C...
4,1047000,0.0,-70.16213,44.98744,9.049562e+08,379.779978,NaN,NaN,310157,RawCAMELS,...,914.491471,914.237495,913.814091,-0.004558,0.005931,0.005651,0.005185,0.004558,POLYGON ((-70.10847473135688 45.21669387774366...,/glade/work/guoqiang/CTSM_cases/CAMELS_Calib/C...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
622,14309500,0.0,-123.74216,42.83017,2.263143e+08,737.623894,NaN,NaN,117484,HSTDX,...,226.552774,225.061137,225.165969,0.006199,0.007259,0.000628,0.001094,0.000628,POLYGON ((-123.85250788390906 42.8720082901822...,/glade/work/guoqiang/CTSM_cases/CAMELS_Calib/C...
623,14316700,0.0,-122.64677,43.45290,5.880250e+08,947.553082,NaN,NaN,176861,RawCAMELS,...,423.559333,423.145281,423.447124,0.000213,-0.279538,-0.280243,-0.279729,0.000213,POLYGON ((-122.47814178484812 43.4038314818071...,/glade/work/guoqiang/CTSM_cases/CAMELS_Calib/C...
624,14325000,0.0,-124.02763,42.79249,4.449257e+08,649.241611,NaN,NaN,166110,MHTDX,...,444.781301,442.202121,443.224396,0.004188,0.003862,-0.001959,0.000348,0.000348,POLYGON ((-124.14601615051636 42.6854521994352...,/glade/work/guoqiang/CTSM_cases/CAMELS_Calib/C...
625,14362250,0.0,-123.12535,42.16583,4.387790e+07,939.577778,NaN,NaN,44972,HSTDX,...,44.118841,41.550384,41.552287,0.059341,0.065158,0.003148,0.003194,0.003148,POLYGON ((-123.12091081349952 42.1398951383789...,/glade/work/guoqiang/CTSM_cases/CAMELS_Calib/C...
